In [23]:
import os
import numpy as np
import pandas as pd
import ast
from collections import defaultdict
import uuid
from tqdm import tqdm
import json

In [24]:
excel_paths = "/home/tanluuuuuuu/Desktop/luunvt/direct_indirect/data/final_luu_a/final_luu/"

merged_df = pd.DataFrame()

for file_name in os.listdir(excel_paths):
    df_file = pd.read_excel(os.path.join(excel_paths, file_name))
    if merged_df.empty: 
        merged_df = df_file
    else:
        merged_df = pd.concat([merged_df, df_file])

merged_df.head()

,asin,product_overview,description,product_detail
0,B0B7NJHF7P,"{'Brand': 'Sophia & William', 'Color': 'As Pic...",['【Sturdy & Durable】The 3-drawer dresser with ...,"{'Brand': 'Sophia & William', 'Color': 'As Pic..."
1,B0159RJYRI,"{'Product Dimensions': '24""D x 12""W x 24""H', '...","['Blackened metal mission slats', 'Cinched met...","{'ASIN': 'B0159RJYRI', 'Date First Available':..."
2,B0BWHLZ2XL,"{'Product Dimensions': '15.75""D x 18.11""W x 18...",['Rose shape handle design: fashionable handle...,"{'ASIN': 'B0BWHLZ2XL', 'Date First Available':..."
3,B09ZFJXS92,"{'Product Dimensions': '12""D x 16""W x 9""H', 'C...",['Material : Engineered Wood. Assembly require...,"{'Product Dimensions': '12""D x 16""W x 9""H', 'C..."
4,B08CBWNZGX,"{'Product Dimensions': '11.8""D x 23.6""W x 24""H...","['Perfect Size: 23.6""L x 11.8""W x 24""H in size...","{'Product Dimensions': '11.8""D x 23.6""W x 24""H..."


In [88]:
random_df = merged_df.sample(n=1500).dropna()
random_df

,asin,product_overview,description,product_detail
84,B0BDY8VKL7,"{'Brand': 'Laojin Balhen', 'Breed Recommendati...",['🐶【Support and Piping Design 】:Supportive rai...,{'Product Dimensions': '25 x 21 x 7 inches; 4 ...
158,B0B331H4KM,"{'Material': 'Polyurethane', 'Color': 'Black',...",['Quality edges on a sturdy material: All four...,"{'Manufacturer': 'Houssity Inc.', 'ASIN': 'B0B..."
46,B007WE32CU,"{'Material': 'Teak', 'Brand': 'Le Click', 'Ite...","['Teak Interlocking Flooring Tiles', 'Windmill...","{'ASIN': 'B007WE32CU', 'Date First Available':..."
150,B09M61N3LV,"{'Brand': 'ClosetMaid', 'Target Species': 'Cat...",['TWO-IN-ONE PET CRATE AND END TABLE: Stylish ...,{'Product Dimensions': '32.4 x 23.4 x 25.2 inc...
14,B07D5P39HY,"{'Size': 'Fits All Speed Bags', 'Brand': 'Meis...",['Super smooth 360° ball bearing allows speed ...,"{'ASIN': 'B07D5P39HY', 'Date First Available':..."
...,...,...,...,...
90,B09M71TKDH,"{'Brand': 'BFPETHOME', 'Breed Recommendation':...","[""Durable Padding Support: the crate bed fille...",{'Product Dimensions': '35 x 23 x 3.5 inches; ...
136,B006ZYU8B6,"{'Size': 'X-Small', 'Sport': 'Boxing', 'Brand'...",['This is an authentic Cleto Reyes speed bag h...,"{'ASIN': 'B01BKZEGXU', 'Date First Available':..."
179,B08RXZ6FDQ,"{'Size': '50*50*160cm', 'Sport': 'Boxing', 'Br...",['【Perfect Gift】Boxing speed ball is a fitness...,{'Package Dimensions': '20.47 x 19.29 x 9.06 i...
159,B001THX6JK,"{'Material': 'Plastic', 'Brand': 'Norsk', 'Ite...","['18.3"" x 18.3"" x .25""(6.3mm)']","{'ASIN': 'B001THX6JK', 'Date First Available':..."


In [121]:
descriptions = random_df['description'].to_numpy()
for idx_des, description in enumerate(descriptions):
    description = description.strip()
    arr_description = ast.literal_eval(description)

    for i in range(len(arr_description)):
        single_description = arr_description[i].lower().strip()

        # This description contains space between the first and second letter.
        # special_description = '''
        # ☀️【large size】- the size of the chair is w23.2 "* l21" * h40 ", the sitting height is 17", and they also have a 40 "high back and a 29" sitting depth. compared with most chairs on the market, we have increased the back, widened the arm spacing, and lengthened the depth of the seat. you have a stronger sense of space when sitting, and more space for activities.
        # '''.strip()

        new_description = []
        last_special = -1
        for idx, letter in enumerate(single_description):
            if not (('a' <= letter and letter <= 'z') or ('0' <= letter and letter <= '9') or letter == ' '):
                pretext = single_description[last_special + 1:idx].strip()
                if pretext != '' and pretext != ' ':
                    new_description.append(pretext)
                new_description.append(letter.strip())
                last_special = idx
            if idx == len(single_description) - 1:
                new_description.append(single_description[last_special + 1:idx + 1].strip())

        arr_description[i] = ' '.join(new_description).strip()
    descriptions[idx_des] = arr_description

In [123]:
# f = open("../data/random_descriptions.txt", 'w')
# for des in descriptions:
#     for line in des:
#         f.write(line + "\n")

In [22]:
from transformers import pipeline
from collections import defaultdict

model_checkpoint = "./distilbert-base-uncased_v11_30e"
token_classifier = pipeline(
    "ner", model=model_checkpoint, aggregation_strategy="simple"
)

In [23]:
f = open("../data/random_descriptions.txt", 'r')
lines = f.readlines()

data_to_import = []

for line in tqdm(lines):
    text_dict = defaultdict()
    line = line.strip()

    text_dict['data'] = {'text': line}

    result_list = []
    result = token_classifier(line)
    for res in result:
        if res['word'][0] == '#':
            continue
        new_id = str(uuid.uuid4())
        res_dict = {
            'id': new_id,
            "from_name": "label",
            "to_name": "text",
            "type": "labels",
            "value": {
              "start": res['start'],
              "end": res['end'],
              "text": res['word'],
              "labels": [
                res['entity_group']
              ]
            }
        }
        result_list.append(res_dict)

    predictions = {
        "model_version": "distilbert-base-uncased_v11_30e",
        "score": '0.5',
        'result': result_list
    }

    text_dict['predictions'] = [predictions]

    data_to_import.append(text_dict)

100%|██████████| 6066/6066 [01:39<00:00, 61.00it/s]


In [24]:
with open('../data/data_random_predictions.json', 'w', encoding='utf-8') as f:
    json.dump(data_to_import, f, ensure_ascii=False, indent=4)

In [16]:
from collections import Counter

f_conll = open("../data/label_studio_2_v2_1044.conll", 'r')
lines = f_conll.readlines()
c = Counter()
for line in lines:
    splt_line = line.strip().split(" ")
    if len(splt_line) != 4:
        continue
    label = splt_line[3]
    if label[0] == 'B':
        if label not in c:
            c[label] = 0
        c[label] += 1
c

Counter({'B-PROPERTY': 1788,
         'B-TARGET_USER': 387,
         'B-MAIN_PRODUCT': 650,
         'B-MAT': 350,
         'B-SHAPE': 9,
         'B-WEIGHT': 96,
         'B-DIMENSION': 289,
         'B-COMPONENT': 252,
         'B-COLOR': 59,
         'B-NMAT': 75})

In [22]:
from collections import Counter

f_conll = open("../data/label_studio_2_v2_1044_resampled.conll", 'r')
lines = f_conll.readlines()
c = Counter()
for line in lines:
    splt_line = line.strip().split(" ")
    if len(splt_line) != 2:
        continue
    label = splt_line[1]
    if label[0] == 'B':
        if label not in c:
            c[label] = 0
        c[label] += 1
c

Counter({'B-PROPERTY': 7184,
         'B-TARGET_USER': 1883,
         'B-MAIN_PRODUCT': 2795,
         'B-MAT': 1724,
         'B-SHAPE': 64,
         'B-WEIGHT': 636,
         'B-DIMENSION': 1525,
         'B-COMPONENT': 1196,
         'B-COLOR': 399,
         'B-NMAT': 408})

In [130]:
import json

f = open("../data/label_studio_1017_shorter.json")

def adding_space(s):
    single_description = s.lower().strip()
    new_description = []
    last_special = -1
    for idx, letter in enumerate(single_description):
        if not (('a' <= letter and letter <= 'z') or ('0' <= letter and letter <= '9') or letter == ' '):
            pretext = single_description[last_special + 1:idx].strip()
            if pretext != '' and pretext != ' ':
                new_description.append(pretext)
            new_description.append(letter.strip())
            last_special = idx
        if idx == len(single_description) - 1:
            new_description.append(single_description[last_special + 1:idx + 1].strip())

    new_description = ' '.join(new_description).lower().strip()
    return new_description

list_data = json.load(f)
list_sentence = []
list_locs = []
list_words = []
list_id = []
for i_list_data, data in enumerate(list_data):
    original_description = data['data']['text']
    original_annotations = data['annotations']

    new_description = adding_space(original_description)

    locs = []
    words = []
    set_new_locations = set()
    for annotation in original_annotations:
        results = annotation['result']
        for ia, result in enumerate(results):
            label = result['value']['labels'][0]
            org_start = result['value']['start']
            org_end = result['value']['end']
            org_word = result['value']['text'].lower().strip()
            new_word = adding_space(org_word)
            word_len = len(new_word)
            new_start = None
            new_end = None
            for i in range(org_start, len(new_description)):
                if new_description[i:i+word_len] == new_word and (i, i+word_len) not in set_new_locations:
                    new_start = i
                    new_end = i + word_len
                    set_new_locations.add((new_start, new_end))
                    list_data[i_list_data]['annotations'][0]['result'][ia]['value']['text'] = new_word
                    list_data[i_list_data]['annotations'][0]['result'][ia]['value']['start'] = new_start
                    list_data[i_list_data]['annotations'][0]['result'][ia]['value']['end'] = new_end
                    break
    list_data[i_list_data]['data']['text'] = new_description
    if len(set_new_locations) != len(annotation['result']):
        # print(i_list_data)
        # print(original_description)
        # print(new_description)
        # display(original_annotations)
        # print(set_new_locations)
        # display(annotation['result'])
        break

In [134]:
with open("../data/label_studio_1017_shorter_fixed.json", "w") as outfile:
    json.dump(list_data, outfile)